# Información General

Este código utiliza las siguientes bibliotecas de Machine Learning:

- **pandas**: Para la manipulación de datos tabulares.
- **scikit-learn**: Para el cálculo de similitud.

## Columnas Relevantes

Se utilizan tres columnas clave del conjunto de datos:

- `user_id`: Identificador único de usuario.
- `item_name`: Nombre único del juego.
- `playtime_forever`: Tiempo jugado por el usuario en un juego.

## Motivo de `cosine_similarity`

Se utiliza `cosine_similarity` de scikit-learn debido a su capacidad para calcular la similitud entre perfiles de manera eficaz. En el contexto de recomendaciones, esto ayuda a encontrar juegos similares en función de las preferencias de los usuarios.

## Pivoteo de Datos y Matriz

Los datos se pivotan para crear una matriz donde:

- Las filas representan juegos únicos.
- Las columnas representan usuarios únicos.
- Los valores en la matriz son el tiempo jugado por un usuario en un juego específico.

El pivoteo permite comparar y calcular la similitud entre juegos y usuarios en función de sus interacciones.



In [6]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Recomendación de Juegos Basada en Similitud de Usuarios

Este método de recomendación sugiere juegos personalizados para usuarios según sus preferencias y similitud con otros usuarios.

## Descripción
La función utiliza la similitud del coseno para calcular relaciones entre usuarios y juegos, identificando los más afines y recomendando títulos similares.

## Parámetros de Entrada
- `id_usuario` (int): El ID único del usuario para el que se desean hacer las recomendaciones.

## Resultado
- `lista_recomendaciones` (list): Una lista de 5 juegos recomendados para el usuario.

## Nota
Este código se basa en una muestra de datos de usuarios e ítems cargados desde un archivo CSV llamado 'new_users_item.csv'. Asegúrate de tener este archivo en el mismo directorio que este código o ajusta la ruta del archivo según sea necesario.


In [8]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Definir la función recomendacion_usuario con muestreo de datos
def recomendacion_usuario(id_usuario):
    # Cargar el archivo CSV
    df1 = pd.read_csv('new_users_item.csv')

    # Muestra aleatoria de datos (ajusta el tamaño según sea necesario)
    df_sample = df1.sample(n=1000, random_state=42)  # Por ejemplo, tomar una muestra de 1000 filas

    # Eliminar duplicados en df_sample si es necesario
    df_sample = df_sample.drop_duplicates(subset=['user_id', 'item_name'])

    df_sample['user_id_numerico'] = range(len(df_sample))

    # Pivotear datos
    matriz_usuarios_items = df_sample.pivot(index='user_id_numerico', columns='item_name', values='playtime_forever').fillna(0)

    # Calcular similitud coseno entre usuarios
    similitudes = cosine_similarity(matriz_usuarios_items)

    # Obtener los índices de los usuarios más parecidos
    indices_similares = similitudes[id_usuario].argsort()[-6:][::-1]

    # Quitar el índice del usuario actual
    indices_similares = indices_similares[1:]

    # Obtener items de usuarios similares
    items_similares = matriz_usuarios_items.iloc[indices_similares] 

    # Obtener items no interactuados por el usuario
    items_no_interactuados = items_similares.columns[~items_similares.columns.isin(matriz_usuarios_items.columns[matriz_usuarios_items.loc[id_usuario] > 0])]

    # Ordenar items por promedio de tiempo en usuarios similares
    items_recomendados = items_similares[items_no_interactuados].mean().sort_values(ascending=False)

    # Devolver las top 5 recomendaciones
    return list(items_recomendados.index)[:5]

# Llamar a la función con el ID de usuario deseado (reemplaza 'id_de_usuario' con el ID real)
id_de_usuario = 520  # Reemplaza con el ID de usuario que desees
recomendaciones = recomendacion_usuario(id_de_usuario)

# Imprimir las recomendaciones
print("Recomendaciones para el usuario", id_de_usuario, ":")
for i, juego in enumerate(recomendaciones, 1):
    print(f"{i}. {juego}")


Recomendaciones para el usuario 520 :
1. 12 Labours of Hercules
2. Rust
3. Rising Storm/Red Orchestra 2 Multiplayer
4. Risk of Rain
5. Robocraft


# Recomendación de Juegos Basada en Similitud de Ítems

Esta función recomienda juegos similares a uno dado a través de la similitud del coseno entre perfiles de juegos.

## Descripción
La función toma un ID de juego como entrada y encuentra juegos similares en función de las preferencias de los usuarios. Utiliza la similitud coseno para calcular la similitud entre perfiles de juegos y devuelve una lista de juegos recomendados.

## Parámetros de Entrada
- `id_juego` (int): El ID del juego para el que se desean realizar las recomendaciones.

## Resultado
- `items_similares` (list): Una lista de 5 juegos recomendados similares al juego con el ID proporcionado.

## Nota
Este código se basa en una muestra de datos de usuarios e ítems cargados desde un archivo CSV llamado 'new_users_item.csv'. Asegúrate de tener este archivo en el mismo directorio que este código o ajusta la ruta del archivo según sea necesario.


In [26]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def recomendacion_juego(id_juego):
    # Leer datos
    df1 = pd.read_csv('new_users_item.csv')

    df_sample = df1.sample(n=1000, random_state=42)
    
    # Eliminar duplicados en df_sample si es necesario
    df_sample = df_sample.drop_duplicates(subset=['item_name', 'user_id'])

    # Pivotear datos
    matriz_items = df_sample.pivot(index='item_name', columns='user_id', values='playtime_forever').fillna(0)

    # Calcular similitud coseno entre items
    similitudes = cosine_similarity(matriz_items)

    # Obtener índices de items más similares
    indices_similares = similitudes[id_juego].argsort()[-6:][::-1]

    # Quitar el índice del item actual
    indices_similares = indices_similares[1:]

    # Obtener los IDs de los ítems similares
    items_similares = list(matriz_items.index[indices_similares][:5])

    return items_similares

  
  
# Llamada a la función con un identificador de juego válido
id_juego = 80  # Reemplaza con un ID de juego real
recomendaciones = recomendacion_juego(id_juego)

# Imprime las recomendaciones
print("Recomendaciones para el juego con ID", id_juego, ":")
for i, juego_id in enumerate(recomendaciones, 1):
    print(f"{i}. Juego ID: {juego_id}")



Recomendaciones para el juego con ID 80 :
1. Juego ID: theHunter
2. Juego ID: FlatOut 2
3. Juego ID: Free to Play
4. Juego ID: Fractal: Make Blooms Not War
5. Juego ID: Forsaken World 


# Funciones de Recomendación de Juegos

## Función `recomendacion_usuario`
Esta función toma el ID de un usuario como entrada y devuelve una lista de 5 juegos recomendados para ese usuario. Utiliza la similitud coseno entre perfiles de usuario y juegos para personalizar las recomendaciones.

<font color="blue"><h2>Función `recomendacion_juego`</h2></font>
Esta función toma el ID de un juego como entrada y devuelve una lista de 5 juegos similares al juego proporcionado. Utiliza la similitud coseno entre perfiles de juegos para identificar juegos relacionados.

Estas funciones utilizan las bibliotecas de pandas y scikit-learn, junto con la información de columnas relevantes y el proceso de pivoteo de datos que se mencionaron anteriormente para proporcionar recomendaciones de juegos personalizadas y basadas en similitud.

